# A first look at data

date: 12/04/2018

Example code for reading and checking data


In [1]:
import pandas as pd
import dask.dataframe as dd
import glob as glob
import numpy as np
from scipy.stats import kde

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.colors import LogNorm
plt.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

This is the part that reads data and makes a basic selection

In [ ]:
df = pd.DataFrame()
df = pd.read_hdf('/data/IceCube/merged2015.h5', key='data')
df['cos2'] = np.cos(df.zenith)**2
df['I'] = 0
df.drop(df.columns.difference(['s125',"cos2", "I"]), 1, inplace= True)
df= df.loc[df.s125>25]
df= df.loc[df.cos2>0.5]  
print(df.count())
#df.drop(["Run", "Event"], 1, inplace = True)


In [ ]:
#hist = plt.hist(np.degrees(df.zenith), bins=150)

hist= plt.hist(np.log10(df.s125),bins= 200)

In [ ]:

import os
filepath='/home/ioana/src/icecube/Noemi/stage/cic_att' 
os.chdir(filepath)
%run data_functions.py
from data_functions import set_intensity
bins = 10
df.reset_index(inplace=True)
df = set_intensity(df, bins)


In [ ]:
%run data_functions.py
from data_functions import get_data_to_fit
#plt.plot(np.log10(df.s125),np.log10(df.I), 'ro')
df['s125_error'] = 0.1*df['s125']
values = get_data_to_fit(df, intensity= 300, n_bins = bins)
print(values)
plt.plot(values[1], values[0], 'ro')

In [ ]:
fit_vals, cov = get_attenuation_parameters(values[0], values[1])
fig2, ax2 =plt.subplots(figsize=(10,6))
plt.grid()
from data_functions import get_s125 
a1=fit_vals[0]
a2=fit_vals[1]
a3=fit_vals[2]
y=get_s125(values[1], a1,a2,a3)
#y2=get_s125(bins, alpha, beta, a3)

ax2.plot(values[1], values[0], lw=0, marker='o')
ax2.plot(values[1], y, color="r")
#ax2.plot(bins2,y2)

In [ ]:
%run data_functions.py

from data_functions import get_attenuation_parameters2

params_scipy, cov2 = sp.optimize.curve_fit(get_s125, values[1], values[0])
# Use as guess
a_true = params_scipy[0]
b_true = params_scipy[1]
f_true = params_scipy[2]

#final_parameters, cov2, sample = get_attenuation_parameters2(s125_fit, s38_fit, s125_fit_error, bins, performMCMC=True)
sample = get_attenuation_parameters2(params_scipy, values[0], values[2], values[1])

In [ ]:
import math
cos_ref = np.cos(math.radians(38))**2
cos2 = values[1] - cos_ref
print(len(sample))
# Plot a subset of the samples
for a, b, f in sample[np.random.randint(len(sample), size=200)]:
    plt.plot(cos2, f * (b * cos2**2 + a * cos2 + 1), color="k", alpha=0.1)
    plt.plot(cos2, f_true * (b_true * cos2**2 + a_true * cos2 + 1), color="r", lw=2, alpha=0.8)
    plt.errorbar(values[1]-cos_ref, values[0], yerr=values[2], fmt=".k")

In [ ]:
import corner
fig = corner.corner(sample, labels=["$a$","$b$", "$s38$"])
fig.savefig("")

In [ ]:
filenames = glob.glob('/data/IceCube/2015/*')
number_of_files = 10000
store = merge_and_select_data(filenames, '/data/IceCube/merged2015.h5', number_of_files )        
dataAll = pd.DataFrame() 
dataAll = store.select('data')
print(dataAll.count())
store.close()    


